# 🧠 Entrenamiento CNN en Google Colab
Este notebook permite entrenar un modelo CNN directamente en Google Colab usando imágenes almacenadas en tu Google Drive.

Incluye:
- Montaje de Google Drive
- Limpieza de imágenes corruptas
- Preprocesamiento con `ImageDataGenerator`
- Entrenamiento del modelo CNN
- Visualización de métricas
- Guardado del modelo entrenado en tu Drive

In [25]:
# 📂 Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
# 📁 Ruta al dataset y al modelo
# Asegúrate de que tu estructura sea por carpetas: drive/MyDrive/mi_proyecto/dataset/train/happy, sad, etc.
dataset_path = '/content/drive/MyDrive/mi_proyecto/dataset/train'
modelo_path = '/content/drive/MyDrive/mi_proyecto/modelo_emociones.h5'
modelo_pathk = '/content/drive/MyDrive/mi_proyecto/modelo_emociones.keras'

In [3]:
# 🧹 Limpieza de imágenes corruptas
import os
from PIL import Image

def limpiar_imagenes_invalidas(ruta):
    for root, _, files in os.walk(ruta):
        for f in files:
            if f.lower().endswith((".png", ".jpg", ".jpeg")):
                path = os.path.join(root, f)
                try:
                    img = Image.open(path)
                    img.verify()
                except Exception as e:
                    print(f"🗑️ Eliminada: {path} — Motivo: {e}")
                    os.remove(path)

limpiar_imagenes_invalidas(dataset_path)

In [32]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.05,
    height_shift_range=0.05,
    zoom_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)



train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(64, 64),
    class_mode='categorical',
    subset='training'
)


val_generator = val_datagen.flow_from_directory(
    dataset_path,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 22968 images belonging to 7 classes.
Found 5741 images belonging to 7 classes.


In [33]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization


num_clases = train_generator.num_classes

model = Sequential([
    Input(shape=(64, 64, 3)),
    Conv2D(32, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_clases, activation='softmax')
])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)              │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 29, 29, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 128)            │     1,605,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,626,311 (6.20 MB)

 Trainable params: 1,626,183 (6.20 MB)

 Non-trainable params: 128 (512.00 B)

In [34]:
from sklearn.utils import class_weight
import numpy as np

class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)

class_weights_dict = dict(enumerate(class_weights))


In [35]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=15,
    class_weight=class_weights_dict
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
718/718 ━━━━━━━━━━━━━━━━━━━━ 66s 89ms/step - accuracy: 0.2392 - loss: 2.0468 - val_accuracy: 0.3107 - val_loss: 1.8528
Epoch 2/15
718/718 ━━━━━━━━━━━━━━━━━━━━ 62s 86ms/step - accuracy: 0.3720 - loss: 1.6014 - val_accuracy: 0.3052 - val_loss: 1.9368
Epoch 3/15
718/718 ━━━━━━━━━━━━━━━━━━━━ 61s 85ms/step - accuracy: 0.4215 - loss: 1.4560 - val_accuracy: 0.4168 - val_loss: 1.5521
Epoch 4/15
718/718 ━━━━━━━━━━━━━━━━━━━━ 61s 85ms/step - accuracy: 0.4779 - loss: 1.2793 - val_accuracy: 0.2836 - val_loss: 1.9067
Epoch 5/15
718/718 ━━━━━━━━━━━━━━━━━━━━ 62s 86ms/step - accuracy: 0.5316 - loss: 1.1203 - val_accuracy: 0.3968 - val_loss: 1.6897
Epoch 6/15
718/718 ━━━━━━━━━━━━━━━━━━━━ 60s 84ms/step - accuracy: 0.5893 - loss: 0.9669 - val_accuracy: 0.4456 - val_loss: 1.6801
Epoch 7/15
718/718 ━━━━━━━━━━━━━━━━━━━━ 60s 83ms/step - accuracy: 0.6602 - loss: 0.8042 - val_accuracy: 0.3843 - val_loss: 1.8595
Epoch 8/15
718/718 ━━━━━━━━━━━━━━━━━━━━ 59s 83ms/step - accuracy: 0.7375 - loss: 0.6433 - 

In [ ]:
# 💾 Guardar el modelo entrenado en Google Drive
model.save(modelo_path)
model.save(modelo_path)

In [ ]:
# 📊 Visualización de métricas
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='Entrenamiento')
plt.plot(history.history['val_accuracy'], label='Validación')
plt.title('Precisión')
plt.xlabel('Época')
plt.ylabel('Precisión')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='Entrenamiento')
plt.plot(history.history['val_loss'], label='Validación')
plt.title('Pérdida')
plt.xlabel('Época')
plt.ylabel('Pérdida')
plt.legend()
plt.show()